<a href="https://colab.research.google.com/github/ohadbarr1/Thesis/blob/main/POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and initializations

Install chronos

In [ ]:
pip install git+https://github.com/amazon-science/chronos-forecasting.git

  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-67tm6yd5
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-67tm6yd5
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit eb7bdfc047de3e7af972b4ee7cf23a7968b7daa3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12

## **Import libraries**

In [ ]:
# Import relevant libraries
import pandas as pd
import numpy as np
from google.colab import files

# Gather financial data
import yfinance as yf

# Plots
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Models
import torch
from sklearn.model_selection import train_test_split
from chronos import ChronosPipeline

## Initializing pipeline

In [ ]:
# Use the Chronos-t5-base model, 200m parameters
pipeline = ChronosPipeline.from_pretrained(
  "amazon/chronos-t5-base",
  device_map="cuda",
  torch_dtype=torch.bfloat16,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/806M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

### Gather VIX Data

In [ ]:
# Data fetching and preprocessing
tickers = ['^VIX9D', '^VIX', '^VIX3M', '^VIX6M']
data = yf.download(tickers, start="2011-01-03")['Close']
# Reordering columns and renaming them according to your format
data = data[['^VIX', '^VIX3M', '^VIX6M', '^VIX9D']]
data.columns = ['VIX', 'VIX3M', 'VIX6M', 'VIX9D']
# Resetting index to make 'Date' a column
data.reset_index(inplace=True)

data



[*********************100%%**********************]  4 of 4 completed


,Date,VIX,VIX3M,VIX6M,VIX9D
0,2011-01-03,17.610001,20.620001,23.400000,16.040001
1,2011-01-04,17.379999,20.610001,23.190001,16.059999
2,2011-01-05,17.020000,20.049999,22.780001,15.570000
3,2011-01-06,17.400000,20.350000,22.870001,15.710000
4,2011-01-07,17.139999,20.290001,22.920000,15.010000
...,...,...,...,...,...
3418,2024-08-05,38.570000,33.709999,30.280001,42.790001
3419,2024-08-06,27.709999,26.760000,25.559999,28.930000
3420,2024-08-07,27.850000,27.010000,25.750000,29.260000
3421,2024-08-08,23.790001,24.129999,23.500000,23.900000


In [ ]:
file_path = "C:/University/Thesis"
data.to_csv('data.csv', index=False)

files.download('data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
VIX9D = data[['Date','VIX9D']]
VIX = data[['Date','VIX']]
VIX3M = data[['Date','VIX3M']]
VIX6M = data[['Date','VIX6M']]


In [ ]:
VIX

,Date,VIX
0,2011-01-03,17.610001
1,2011-01-04,17.379999
2,2011-01-05,17.020000
3,2011-01-06,17.400000
4,2011-01-07,17.139999
...,...,...
3418,2024-08-05,38.570000
3419,2024-08-06,27.709999
3420,2024-08-07,27.850000
3421,2024-08-08,23.790001


In [ ]:
context = torch.tensor(VIX["VIX"])
prediction_length = 7
forecast = pipeline.predict(context, prediction_length)

In [ ]:
forecast

tensor([[[18.2013, 17.4157, 17.1538, 17.0228, 18.2013, 18.4632, 16.4991],
         [19.2489, 18.7251, 18.2013, 18.4632, 17.9394, 17.2847, 16.8919],
         [18.2013, 18.4632, 20.4274, 22.7844, 21.7368, 23.1772, 21.2131],
         [17.2847, 17.0228, 15.7134, 14.7968, 14.7968, 13.7492, 13.7492],
         [16.7609, 17.2847, 17.0228, 17.0228, 16.2372, 15.1896, 14.1420],
         [20.2965, 18.3323, 18.4632, 17.2847, 16.3681, 16.3681, 16.3681],
         [18.4632, 18.4632, 18.7251, 21.4750, 21.8678, 21.4750, 19.7727],
         [18.3323, 17.2847, 17.6776, 16.8919, 16.6300, 17.0228, 16.7609],
         [19.1180, 18.2013, 18.4632, 18.8561, 18.9870, 18.7251, 19.9036],
         [18.2013, 18.2013, 19.7727, 19.2489, 19.3798, 19.5108, 19.6417],
         [18.0704, 17.2847, 16.1062, 14.6658, 14.7968, 14.0111, 14.2730],
         [19.9036, 20.2965, 20.8202, 18.0704, 17.1538, 16.8919, 14.6658],
         [19.3798, 18.5942, 18.7251, 18.3323, 17.8085, 16.7609, 17.4157],
         [18.8561, 15.9753, 16.1062, 1

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Assuming VIX contains your historical data and forecast is the predicted output
forecast_index = list(range(len(VIX), len(VIX) + prediction_length))
low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)

# Create the figure
fig = go.Figure()

# Add the historical data line
fig.add_trace(go.Scatter(
    x=VIX.index,
    y=VIX["VIX"],
    mode='lines',
    name='historical data',
    line=dict(color='royalblue')
))

# Add the median forecast line
fig.add_trace(go.Scatter(
    x=forecast_index,
    y=median,
    mode='lines',
    name='median forecast',
    line=dict(color='tomato')
))

# Add the prediction interval (as a shaded area)
fig.add_trace(go.Scatter(
    x=forecast_index + forecast_index[::-1],
    y=list(high) + list(low)[::-1],
    fill='toself',
    fillcolor='rgba(255, 99, 71, 0.3)',  # tomato color with transparency
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='80% prediction interval'
))

# Update the layout of the figure
fig.update_layout(
    title="Forecast Visualization",
    xaxis_title="Time",
    yaxis_title="VIX",
    legend=dict(x=0, y=1),
    hovermode="x",
    template="plotly_white",
    xaxis=dict(
        gridcolor="LightGray"
    ),
    yaxis=dict(
        gridcolor="LightGray"
    )
)

# Display the interactive plot
fig.show()


Train the model

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

In [ ]:
# Set model to evaluation mode
pipeline.model.eval()

# Predict the next day's VIX value using the last sequence from the validation set
with torch.no_grad():
    last_sequence = torch.tensor(X_val_seq[-1:], dtype=torch.float32).cuda()
    predicted_vix = pipeline.model(last_sequence)

# Inverse transform the prediction to get the original scale
predicted_vix_original_scale = scaler.inverse_transform(predicted_vix.cpu().numpy().reshape(-1, 1))

print(f"Predicted VIX value for the next day: {predicted_vix_original_scale[0][0]}")
